 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 3: Optimisation sans contraintes et méthodes itératives
Tangi Migot et Paul Raynaud

In [1]:
using LinearAlgebra, NLPModels, Printf

In [2]:
#Test problem:
using ADNLPModels
fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
x0H = [10., 20.]
himmelblau = ADNLPModel(fH, x0H)

problem2 = ADNLPModel(x->-x[1]^2, ones(3))

roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]

ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  ForwardDiffADGradient,
  ForwardDiffADHvprod,
  EmptyADbackend,
  EmptyADbackend,
  EmptyADbackend,
  ForwardDiffADHessian,
  EmptyADbackend,
}
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                            

## Commentaires sur Julia

Quelques commentaires sur des morceaux de codes que vous avez vu:
- les structures, exemple [GenericExecutionStats](https://github.com/JuliaSmoothOptimizers/SolverCore.jl/blob/0091f437a26a27ac8aa53d5e37647223722f7f7c/src/stats.jl#L60) (constructeur, attribut, type).


In [3]:
using SolverCore

In [4]:
? GenericExecutionStats

ErrorException: syntax: invalid identifier name "?"

- Les arguments dans les fonctions. Lire attentivement [la documentation Julia sur les fonctions](https://docs.julialang.org/en/v1/manual/functions/) pour comprendre l'utilisation des `Optional Arguments` et des `Keywords Arguments`. Ce type d'arguments est très utile dans nos applictions où les solveurs dépendent de paramètre dont on peut fixer des valeurs par défaut.

## Exercice 1: Méthode BFGS avec mémoire limitée (L-BFGS)

Le but de cet exercice est d'implémenter la méthode BFGS à mémoire limitée vue en cours en utilisant les `InverseLBFGSOperator` du package `LinearOperators.jl`. Il y a aussi un petit exemple dans la documentation du package [LinearOperators.jl/dev/tutorial/#Limited-memory-BFGS-and-SR1](https://juliasmoothoptimizers.github.io/LinearOperators.jl/dev/tutorial/#Limited-memory-BFGS-and-SR1).

In [5]:
using LinearOperators

In [6]:
? InverseLBFGSOperator

ErrorException: syntax: invalid identifier name "?"

ce qui est important dans ce type de méthode est:
- le paramètre mémoire
- la mise à jour de l'opérateur avec la fonction `push!`
- si on a pas une direction de descente, alors on skip
- recherche linéaire d'Armijo

In [7]:
? LinearOperators.push!

ErrorException: syntax: invalid identifier name "?"

In [8]:
function armijo(xk, dk, fk, gk, slope, nlp :: AbstractNLPModel; τ1 = 1.0e-4, t_update = 1.5)
  t = 1.0
  fk_new = obj(nlp, xk + dk) # t = 1.0
  while fk_new > fk + τ1 * t * slope
    t /= t_update
    fk_new = obj(nlp, xk + t * dk)
  end
  return t, fk_new
end

armijo (generic function with 1 method)

In [9]:
function limited_bfgs(nlp      :: AbstractNLPModel;
                      x        :: AbstractVector = nlp.meta.x0,
                      atol     :: Real = √eps(eltype(x)), 
                      rtol     :: Real = √eps(eltype(x)),
                      max_eval :: Int = -1,
                      max_time :: Float64 = 30.0,
                      f_min    :: Float64 = -1.0e16,
                      verbose  :: Bool = true,
                      mem      :: Int = 5)
  start_time = time()
  elapsed_time = 0.0

  T = eltype(x)
  n = nlp.meta.nvar

  xt = zeros(T, n)
  ∇ft = zeros(T, n)

  f = obj(nlp, x)
  ∇f = grad(nlp, x)
#################################################
  H = InverseLBFGSOperator(T,n)
#################################################

  ∇fNorm = norm(∇f) #nrm2(n, ∇f)
  ϵ = atol + rtol * ∇fNorm
  iter = 0

  @info log_header([:iter, :f, :dual, :slope, :bk], [Int, T, T, T, T],
                   hdr_override=Dict(:f=>"f(x)", :dual=>"‖∇f‖", :slope=>"∇fᵀd"))

  optimal = ∇fNorm ≤ ϵ
  unbdd = f ≤ f_min
  tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  stalled = false
  status = :unknown

  while !(optimal || tired || stalled || unbdd)

#################################################
    d = -H*∇f
#################################################
    slope = dot(d, ∇f)
    if slope ≥ 0
      @error "not a descent direction" slope
      status = :not_desc
      stalled = true
      continue
    end

    # Perform improved Armijo linesearch.
    t, ft = armijo(x, d, f, ∇f, slope, nlp)
        
    @info log_row(Any[iter, f, ∇fNorm, slope, t])

    # Update L-BFGS approximation.
    xt = x + t * d
    ∇ft = grad(nlp, xt) # grad!(nlp, xt, ∇ft)
#################################################
    push!(H,xt-x,∇ft-∇f)
#################################################

    # Move on.
    x = xt
    f = ft
    ∇f = ∇ft

    ∇fNorm = norm(∇f) #nrm2(n, ∇f)
    iter = iter + 1

    optimal = ∇fNorm ≤ ϵ
    unbdd = f ≤ f_min
    elapsed_time = time() - start_time
    tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  end
  @info log_row(Any[iter, f, ∇fNorm])

  if optimal
    status = :first_order
  elseif tired
    if neval_obj(nlp) > max_eval ≥ 0
      status = :max_eval
    elseif elapsed_time > max_time
      status = :max_time
    end
  elseif unbdd
        status = :unbounded
  end

  return GenericExecutionStats(
        nlp,
        status=status,
        solution=x,
        objective=f,
        dual_feas=∇fNorm,
        iter=iter,
        elapsed_time=elapsed_time,
    )
end

limited_bfgs (generic function with 1 method)

In [10]:
#Unit/Validation Tests
# Réaliser un test unitaire

In [11]:
include("test-lbfgs")

(stats.status, stats.solution) = (:first_order, [3.584428266659278, -1.8481265666485829])
(stats.status, stats.solution) = 

(:unbounded, [1.29140163e8, 1.0, 1.0])


(stats.status, stats.solution) = (:first_order, [0.9999999887950609, 0.9999999782159007])
(stats.status, stats.solution) = 

(:unbounded, [-975544.6831042847, -764227.9248199855])
(stats.status, stats.solution) = (:first_order, [0.9999999962625671, -3.1168150200102845e-9])
(stats.status, stats.solution) = (:first_order, [0.99999999073849, -6.617373493448244e-9])


Test Summary: | Pass  Total   Time
test set      |    9      9  14.6s


Test.DefaultTestSet("test set", Any[], 9, false, false, true, 1.70812397633908e9, 1.70812399093274e9, false)

### Bonus:

- Compare l'implémentation de `limited_bfgs` avec la fonction `lbfgs` qui est disponible dans `JSOSolvers.jl`.
- On veut pouvoir tester "facilement" plusieurs valeurs de $\tau$ et du paramètre de mise à jour dans `armijo` sur les problèmes tests. Comment modifier le code pour que ça soit possible?

On peut mesurer deux executions de fonctions Julia grâce aux fonctions de `BenchmarkTools.jl`:

In [12]:
using BenchmarkTools
using JSOSolvers

In [13]:
? @time

ErrorException: syntax: invalid identifier name "?"

In [14]:
@time "lbfgs(himmelblau) :" lbfgs(himmelblau)
@time "limited_bfgs(himmelblau) :" limited_bfgs(himmelblau)

lbfgs(himmelblau) :: 3.268267 seconds (1.53 M allocations: 95.301 MiB, 1.68% gc time, 99.95% compilation time)


┌ Info:   iter      f(x)      ‖∇f‖      ∇fᵀd        bk  
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:29
┌ Info:      0   1.7e+05   3.3e+04  -1.1e+09   1.0e-03
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54


limited_bfgs(himmelblau) :: 1.198546 seconds (397.50 k allocations: 25.789 MiB, 1.82% gc time, 99.82% compilation time)


┌ Info:      1   2.7e+04   8.6e+03  -5.9e+04   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      2   1.3e+03   8.6e+02  -6.6e+02   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      3   7.2e+02   5.7e+02  -8.8e+02   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      4   1.6e+02   2.0e+02  -1.6e+02   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      5   4.6e+01   8.6e+01  -5.4e+01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      6   1.0e+01   3.3e+01  -1.3e+01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      7   1.8e+00   1.2e+01  -2.6e+00   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      8   1.9e-01   

"Execution stats: first-order stationary"

## Exercice 2: NewtonCG

Le but de cet exercice est d'adapter les méthodes de Newton de façon à résoudre le système linéaire avec une méthode itérative de type gradient conjugué comme suit ($B_k$ représente la matrice hessienne):

<img src="LineSearchNewtonCG.png" width=600 height=600 />

In [15]:
function cg_optim(H, ∇f)
    #setup the tolerance:
    n∇f = norm(∇f)
#####################################
    ϵk = min(0.5, √(n∇f))*n∇f
####################################
    n = length(∇f)
    z = zeros(n)
    r = ∇f
    d = -r
    
    j = 0
    while norm(r) ≥ ϵk && j < 3 * n
###############################################
        if dot(d, H * d) ≤ 0
            if j == 0
                return -∇f
            else
                return z 
            end
        end
##############################################
        α = dot(r,r)/dot(d,H*d)
##############################################        
        z += α * d
        nrr2 = dot(r, r)
        r += α * H * d
##############################################
        β  = dot(r, r)/nrr2
##############################################
        d  = -r + β * d
        j += 1
    end
    return z
end

cg_optim (generic function with 1 method)

Ce qui est important ici est qu'on a pas besoin de stocker/évaluer la matrice hessienne entière mais simplement le produit entre la hessienne et un vecteur. Pour un `NLPModels` on utilise:

In [16]:
using NLPModels

In [17]:
? NLPModels.hprod

ErrorException: syntax: invalid identifier name "?"

In [18]:
? NLPModels.hess_op

ErrorException: syntax: invalid identifier name "?"

In [19]:
function armijo_Newton_cg(nlp      :: AbstractNLPModel;
                          x        :: AbstractVector = nlp.meta.x0,
                          atol     :: Real = √eps(eltype(x)), 
                          rtol     :: Real = √eps(eltype(x)),
                          max_eval :: Int = -1,
                          max_time :: Float64 = 30.0,
                          f_min    :: Float64 = -1.0e16)
  start_time = time()
  elapsed_time = 0.0

  T = eltype(x)
  n = nlp.meta.nvar

  f = obj(nlp, x)
  ∇f = grad(nlp, x)
#################################################
  H =  hess_op(nlp,x)
#################################################

  ∇fNorm = norm(∇f) #nrm2(n, ∇f)
  ϵ = atol + rtol * ∇fNorm
  iter = 0

  @info log_header([:iter, :f, :dual, :slope, :bk], [Int, T, T, T, T],
                   hdr_override=Dict(:f=>"f(x)", :dual=>"‖∇f‖", :slope=>"∇fᵀd"))

  optimal = ∇fNorm ≤ ϵ
  unbdd = f ≤ f_min
  tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  stalled = false
  status = :unknown

  while !(optimal || tired || stalled || unbdd)
        
    d = cg_optim(H, ∇f)
        
    slope = dot(d, ∇f)
    if slope ≥ 0
      @error "not a descent direction" slope
      status = :not_desc
      stalled = true
      continue
    end

    # Perform improved Armijo linesearch.
    t, f = armijo(x, d, f, ∇f, slope, nlp)
        
    @info log_row(Any[iter, f, ∇fNorm, slope, t])

    # Update L-BFGS approximation.
    x += t * d
    ∇f = grad(nlp, x)
#################################################
    H = hess_op(nlp,x)
#################################################

    ∇fNorm = norm(∇f) #nrm2(n, ∇f)
    iter = iter + 1

    optimal = ∇fNorm ≤ ϵ
    unbdd = f ≤ f_min
    elapsed_time = time() - start_time
    tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  end
  @info log_row(Any[iter, f, ∇fNorm])

  if optimal
    status = :first_order
  elseif tired
    if neval_obj(nlp) > max_eval ≥ 0
      status = :max_eval
    elseif elapsed_time > max_time
      status = :max_time
    end
  elseif unbdd
        status = :unbounded
  end

  return GenericExecutionStats(nlp, status = status, solution=x, objective=f, dual_feas=∇fNorm,
                               iter=iter, elapsed_time=elapsed_time)
end

armijo_Newton_cg (generic function with 1 method)

In [20]:
#Unit/Validation Tests
# Réaliser un test unitaire

In [21]:
include("test-newton")

(stats.status, stats.solution) = (:first_order, [2.9999993048920612, 2.000003199723837])
(stats.status, stats.solution) = 

(:unbounded, [1.29140163e8, 1.0, 1.0])


(stats.status, stats.solution) = (:first_order, [0.999999476502548, 0.9999989509130072])
(stats.status, stats.solution) = 

(:unbounded, [-475732.37111010315, -259818.976413054])
(stats.status, stats.solution) = (:first_order, [1.000000000013107, 6.553559582523889e-12])
(stats.status, stats.solution) = (:first_order, [1.0000000000000009, 4.489396480695633e-16])
Test Summary: | Pass  Total  Time
test set      |    9      9  3.9s


Test.DefaultTestSet("test set", Any[], 9, false, false, true, 1.708124000100389e9, 1.708124004022779e9, false)

## Comment préparer un benchmark

On veut maintenant pouvoir réaliser un benchmark de plusieurs solveurs. Pour comparer les algorithmes, il nous faut une collection de problèmes tests et on va utiliser `OptimizationProblems.jl`.

In [22]:
using OptimizationProblems, OptimizationProblems.ADNLPProblems

Vous pouvez trouver un tutoriel de comment réaliser un benchmark avec ce package sur la documentation [OptimizationProblems.jl/dev/benchmark/](https://juliasmoothoptimizers.github.io/OptimizationProblems.jl/dev/benchmark/).

Il est fort possible que les petits problèmes tests que l'on résout après l'implémentation ne suffisent pas à déceler des bugs. Mais on peut toujours analyser l'éxecution de notre algorithme sur certains problèmes de la collection afin d'améliorer la valeur de certains paramètres (limite de temps, d'itérations, d'évaluations), détecter un bug, etc.

In [36]:
prbl = OptimizationProblems.ADNLPProblems.AMPGO02()
limited_bfgs(prbl)

┌ Info:   iter      f(x)      ‖∇f‖      ∇fᵀd        bk  
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:29
┌ Info:      0   8.4e-01   3.9e+00  -1.6e+01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      1   2.0e-01   2.4e+00  -1.4e+01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      2  -8.4e-01   9.9e-01  -4.2e+00   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      3  -1.4e+00   2.4e+00  -7.4e+00   1.3e-01
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      4  -1.6e+00   1.7e+00  -2.8e-01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      5  -1.7e+00   2.2e-01  -4.2e-03   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      6  -1.7e+00 

"Execution stats: first-order stationary"

Vous vous en doutez pour le rapport de cette semaine on va vouloir réaliser une benchmark avec les deux méthodes que l'on a codé.

### Appendix:

Une petite remarque sur la gestion de la mémoire:

In [24]:
#Pour les nombres:
a = 1
@show a
b = a
@show b
a = 2
@show b

a = 1
b = 1


b = 1


1

In [25]:
#Pour les tableaux:
a = zeros(Float64, 2) #or zeros(2)
@show a
b = a
@show (a,b)
a = ones(Float64, 2)
@show (a,b)

#Pour les tableaux:
a = ones(Float64, 2)
b = a
@show (a,b)
a .= 2*ones(Float64, 2) #same would go with grad!
@show (a,b)

#Pour les tableaux:
a = ones(Float64, 2)
b = copy(a) # or similar(a)
@show (a,b)
a .= 2 .* ones(Float64, 2) #same would go with grad!
@show (a,b)

a = [0.0, 0.0]
(a, b) = ([0.0, 0.0], [0.0, 0.0])
(a, b) = ([1.0, 1.0], [0.0, 0.0])
(a, b) = ([1.0, 1.0], [1.0, 1.0])


(a, b) = ([2.0, 2.0], [2.0, 2.0])
(a, b) = ([1.0, 1.0], [1.0, 1.0])
(a, b) = ([2.0, 2.0], [1.0, 1.0])


([2.0, 2.0], [1.0, 1.0])

In [26]:
#Pour les NLPModels, il existe aussi des fonctions qui interviennent sur la mémoire
gk = grad(himmelblau, x0H)
grad!(himmelblau, x0H, gk) #équivaut à gk .= grad(nlp, x0)

2-element Vector{Float64}:
  5166.0
 32458.0

Comparez les `grad` et `grad!` à l'aide de `@benchmark` ainsi que:


In [27]:
n = 10000
a = rand(n)
b = ones(n)
c = similar(a)
@benchmark c = 2 * a + b * 1.2

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   19.696 μs …   9.976 ms  ┊ GC (min … max):  0.00% … 98.76%
 Time  (median):     130.495 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   176.621 μs ± 541.503 μs  ┊ GC (mean ± σ):  21.94% ±  7.05%

                      ▃█▅▂                                       
  ▃▂▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▃▅█████▆▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  19.7 μs          Histogram: frequency by time          317 μs <

 Memory estimate: 234.52 KiB, allocs estimate: 6.


et

In [28]:
@benchmark c .= 2 .* a .+ b .* 1.2

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  6.311 μs … 242.935 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.648 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.298 μs ±   3.467 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       █▃▆                                                     
  ▅▄▂▂████▃▇▂▃▅▂▂▂▂▂▁▂▂▂▂▂▃▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  6.31 μs         Histogram: frequency by time          17 μs <

 Memory estimate: 224 bytes, allocs estimate: 6.

En particulier, observez la mémoire allouée et le temps moyen requis pour performer la ligne d'instruction.
La mémoire nécessaire varie en fonction du nombre et du type d'opération effectué.
Parmis les opérations allouant le moins de mémoire, on retrouve:

In [29]:
@benchmark c .= 0

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.430 μs … 179.734 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.670 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.921 μs ±   2.888 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▆█▇▇▅▁                                                     
  ▃▇██████▆▅▄▅▅▄▄▄▄▄▅▅▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▃
  1.43 μs         Histogram: frequency by time         3.7 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Pour manipuler plus finement la mémoire au cours de votre implémentation, vous pouvez utilisez des fonctions de `LinearAlgebra`

In [30]:
? axpy!
? axpby!

ErrorException: syntax: invalid identifier name "?"

qui sont des routines `BLAS`.
Faites attention, ces routines ont des effets de bord sur les structures de données `y`.
Dès lors, l'utilisation de `@benchmark` accumule successivement les effets de bord, d'où l'utilisation des tests avant `@benchmark`.

In [31]:
y = zeros(n)
x = ones(n)
a = 2
axpy!(a, x, y)
@show y == a * x

@benchmark axpy!(a, x, y)

y == a * x = true


BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.240 μs … 342.124 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.677 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.967 μs ±   4.442 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▄▃▄█▃▃ ▂                                                  
  ▅▇▆██████▆█▆▅▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂▂ ▃
  2.24 μs         Histogram: frequency by time         5.8 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [32]:
y = π * ones(n)
x = ones(n)
a = 2
b = 3
axpby!(a, x, b, y)
@show y == a * ones(n) + b * π * ones(n)

@benchmark axpby!(a, x, b, y)

y == a * ones(n) + b * π * ones(n) = true


BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.178 μs … 282.816 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.703 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.969 μs ±   3.432 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▃▂█▂                                                    
  ▆▅▃▄▇████▇▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂ ▃
  2.18 μs         Histogram: frequency by time        6.26 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Cela fontionne aussi pour les matrices

In [33]:
y = π * ones(n,n)
x = ones(n,n)
a = 2
b = 3
axpby!(a, x, b, y)
@show y == a * ones(n,n) + b * π * ones(n,n)

@benchmark axpby!(a, x, b, y)

y == a * ones(n, n) + b * π * ones(n, n) = true


BenchmarkTools.Trial: 41 samples with 1 evaluation.
 Range (min … max):  113.915 ms … 149.446 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     119.989 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   123.039 ms ±   8.901 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄█▁▄ ▄ ▄ ▁   ▄     ▁ ▁                                        
  ▆████▆█▁█▁█▁▆▆█▁▁▁▁▁█▆█▁▁▆▆▁▁▁▆▁▁▁▁▆▆▁▁▆▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▆ ▁
  114 ms           Histogram: frequency by time          149 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

## Projet de laboratoire 3

### Exercice 1 : Méthode quasi-Newton L-BFGS

In [34]:
nlp = OptimizationProblems.ADNLPProblems.AMPGO02()
limited_bfgs(nlp)

┌ Info:   iter      f(x)      ‖∇f‖      ∇fᵀd        bk  
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:29
┌ Info:      0   8.4e-01   3.9e+00  -1.6e+01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      1   2.0e-01   2.4e+00  -1.4e+01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      2  -8.4e-01   9.9e-01  -4.2e+00   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      3  -1.4e+00   2.4e+00  -7.4e+00   1.3e-01
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      4  -1.6e+00   1.7e+00  -2.8e-01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      5  -1.7e+00   2.2e-01  -4.2e-03   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:54
┌ Info:      6  -1.7e+00 

"Execution stats: first-order stationary"

### Exercice 2 : Méthode Newton-CG

In [37]:
armijo_Newton_cg(nlp)

┌ Info:   iter      f(x)      ‖∇f‖      ∇fᵀd        bk  
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:24
┌ Info:      0   2.0e-01   3.9e+00  -1.6e+01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:48


┌ Info:      1  -1.3e-01   2.4e+00  -4.4e+00   3.0e-01
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:48
┌ Info:      2  -3.2e-01   2.0e+00  -4.1e-01   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:48
┌ Info:      3  -3.2e-01   1.6e-01  -2.6e-03   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:48
┌ Info:      4  -3.2e-01   1.1e-03  -1.3e-07   1.0e+00
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:48
┌ Info:      5  -3.2e-01   4.8e-08
└ @ Main /Users/jules/Desktop/MTH8408/Git/MTH8408-Hiv24/lab3/Lab3-notebook.ipynb:65


"Execution stats: first-order stationary"

### Exercice 3

In [ ]:

########################## CRÉATION DU DICTIONNAIRE DE SOLVEURS ##########################
solvers = solvers = Dict(
  :lbfgs1 => model -> lbfgs(model, mem=1),
  :lbfgs5 => model -> lbfgs(model, mem=5),
  :lbfgs20 => model -> lbfgs(model, mem=20),
)


########################## SÉLECTION DES PROBLÈMES POUR LES TEST ##########################
n = 20
ad_problems = (eval(Meta.parse(problem))(;n) for problem ∈ OptimizationProblems.meta[!, :name])


######################### PASSAGE DES PROBLÈMES DANS LES SOLVEURS #########################
stats = bmark_solvers(
  solvers, ad_problems,
  skipif=prob -> (!unconstrained(prob) || get_nvar(prob) > 100 || get_nvar(prob) < 5),
)

In [ ]:
########################## MISE EN FORME DU TABLEAU DES RÉSULTATS #########################
cols = [:id, :name, :nvar, :objective, :dual_feas, :neval_obj, :neval_grad, :neval_hess, :iter, :elapsed_time, :status]
header = Dict(
  :nvar => "n",
  :objective => "f(x)",
  :dual_feas => "‖∇f(x)‖",
  :neval_obj => "# f",
  :neval_grad => "# ∇f",
  :neval_hess => "# ∇²f",
  :elapsed_time => "t",
)

for solver ∈ keys(solvers)
  pretty_stats(stats[solver][!, cols], hdr_override=header, tf=tf_markdown)
end

In [ ]:
########################### CRÉATION DES PROFILS DE PERFORMANCE ##########################

first_order(df) = df.status .== :first_order
unbounded(df) = df.status .== :unbounded
solved(df) = first_order(df) .| unbounded(df)

costnames = ["time", "obj + grad + hess"]
costs = [
  df -> .!solved(df) .* Inf .+ df.elapsed_time,
  df -> .!solved(df) .* Inf .+ df.neval_obj .+ df.neval_grad .+ df.neval_hess,
]

using Plots
gr()

profile_solvers(stats, costs, costnames)